In [1]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

# Invoke a primitive. For more details see https://qiskit.org/documentation/partners/qiskit_ibm_runtime/tutorials.html
# result = Sampler("ibmq_qasm_simulator").run(circuits).result()

1. Load and Preprocess the MNIST Dataset:

We will use the MNIST dataset for training and evaluation.

In [2]:
import numpy as np
from tensorflow.keras.datasets import mnist
from sklearn.preprocessing import StandardScaler

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Flatten the images
x_train = x_train.reshape(-1, 28*28)
x_test = x_test.reshape(-1, 28*28)

# Standardize the data
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)


2. Define the Quantum Variational Autoencoder (QVAE):
We'll design a simple QVAE with a parameterized quantum circuit as the encoder and decoder.

In [3]:
def variational_circuit(params, n_qubits):
    """Generate a quantum state based on parameters."""
    qc = QuantumCircuit(n_qubits)
    for i in range(n_qubits):
        qc.u(params[3*i], params[3*i + 1], params[3*i + 2], i)
    return qc

def encoder(params, data_point):
    """Encode the data point into a quantum state."""
    n_qubits = int(np.ceil(np.log2(len(data_point))))
    qc = QuantumCircuit(n_qubits)
    normalized_data = data_point / np.linalg.norm(data_point)
    qc.initialize(normalized_data, list(range(n_qubits)))
    variational_qc = variational_circuit(params, n_qubits)
    qc.compose(variational_qc, inplace=True)
    return qc

def decoder(params, encoded_state):
    """Decode the quantum state back to classical data."""
    n_qubits = encoded_state.num_qubits
    variational_qc = variational_circuit(params, n_qubits)
    encoded_state.compose(variational_qc.inverse(), inplace=True)
    return encoded_state


3. Training the QVAE:
We will train the QVAE using the MNIST dataset.

In [4]:
from qiskit import Aer, transpile, assemble, execute

def train_qvae(x_train, epochs=100, lr=0.1):
    """Train the QVAE model."""
    n_qubits = int(np.ceil(np.log2(x_train.shape[1])))
    encoder_params = np.random.rand(3 * n_qubits)  # Initial encoder parameters
    decoder_params = np.random.rand(3 * n_qubits)  # Initial decoder parameters
    backend = Aer.get_backend('statevector_simulator')
    
    for epoch in range(epochs):
        for data_point in x_train:
            # Encode the data point
            qc = encoder(encoder_params, data_point)
            
            # Decode the quantum state
            decoded_qc = decoder(decoder_params, qc)
            
            # Compute the loss (mean squared error)
            t_qc = transpile(decoded_qc, backend)
            qobj = assemble(t_qc)
            final_state = execute(decoded_qc, backend).result().get_statevector()
            
            reconstructed_data = np.abs(final_state)**2
            loss = np.mean((data_point - reconstructed_data)**2)
            
            # Simplistic parameter update (gradient-free for simplicity)
            encoder_params -= lr * loss
            decoder_params -= lr * loss
            
    return encoder_params, decoder_params


4. Visualizing the Reconstructed and Generated Data:
We will visualize the reconstructed and generated data.

In [5]:
import matplotlib.pyplot as plt

def visualize_reconstruction(encoder_params, decoder_params, data_point):
    """Visualize the reconstructed data."""
    qc = encoder(encoder_params, data_point)
    decoded_qc = decoder(decoder_params, qc)
    backend = Aer.get_backend('statevector_simulator')
    t_qc = transpile(decoded_qc, backend)
    qobj = assemble(t_qc)
    final_state = execute(decoded_qc, backend).result().get_statevector()
    reconstructed_data = np.abs(final_state)**2
    
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title('Original Data')
    plt.imshow(data_point.reshape(28, 28), cmap='gray')
    plt.subplot(1, 2, 2)
    plt.title('Reconstructed Data')
    plt.imshow(reconstructed_data.reshape(28, 28), cmap='gray')
    plt.show()


5. Putting It All Together and Running the Example:
Here, we'll train our QVAE on the MNIST dataset and visualize the reconstructed data.

In [6]:
# Train the QVAE model
encoder_params, decoder_params = train_qvae(x_train, epochs=100, lr=0.1)

# Visualize the reconstructed data
data_point = x_test[0]  # Use the first test data point for visualization
visualize_reconstruction(encoder_params, decoder_params, data_point)
